In [ ]:
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def noise(seq):
    traj = [] 
    toks = tokenizer(seq, max_length=512, truncation=True)['input_ids'][1:] # remove CLS token
    N = len(toks)
    
    while toks:
        toks = toks[:len(toks)//2]
        traj.append(tokenizer.decode(toks))
        
    return traj[::-1], 0, N

In [ ]:
import os
import json
from tqdm import tqdm

lengths = []

with open('../data/imdb/imdb_train_1.jsonl', 'w') as train_f, open('../data/imdb/imdb_dev_1.jsonl', 'w') as dev_f:
    for i, file in tqdm(enumerate(os.listdir('../data/imdb/raw/train/neg'))):
        df = train_f if i < 10000 else dev_f
        with open(f'../data/imdb/raw/train/neg/{file}', 'r') as f:
            seq = f.read()
            *traj, N = noise(seq)
            json.dump(traj, df)
            df.write('\n')
            lengths.append(N)
            
    for i, file in tqdm(enumerate(os.listdir('../data/imdb/raw/train/pos'))):
        df = train_f if i < 10000 else dev_f
        with open(f'../data/imdb/raw/train/pos/{file}', 'r') as f:
            seq = f.read()
            *traj, N = noise(seq)
            json.dump(traj, df)
            df.write('\n')
            lengths.append(N)

In [ ]:
import json
import torch
import pickle
from transformers import BertTokenizer
from constants import *
from tqdm import tqdm

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

with open('../data/imdb/imdb_train_1.jsonl', 'r') as f:
    for i, line in tqdm(enumerate(f.readlines())):
        if not line: continue
        traj, _ = json.loads(line)
        ids = tokenizer(traj)['input_ids']
        
        traj_op_tgts = []
        traj_tok_tgts = []
        traj_idx_tgts = []
        
        for j, seq in enumerate(ids[1:], start=1):
            N = len(ids[j-1]) - 2
            M = len(seq) - 2 - N
            
            op_tgts = [INS_ID] + [CPY_ID for _ in range(N)] + [INS_ID for _ in range(M)] + [EOS_ID]
            tok_tgts = [BOS_TOKEN_ID] + [PAD_TOKEN_ID for _ in range(N)] + [tok for tok in seq[1+N:1+N+M]] + [PAD_TOKEN_ID]
            idx_tgts = [0]  + [i for i in range(1, N+1)] + [0 for _ in range(M)] + [0]
           
            # pad out to 512 
            op_tgts += [PAD_ID for _ in range(512-len(seq))]
            tok_tgts += [PAD_TOKEN_ID for _ in range(512-len(seq))]
            idx_tgts += [0 for _ in range(512-len(seq))]
            
            traj_op_tgts.append(op_tgts)
            traj_tok_tgts.append(tok_tgts)
            traj_idx_tgts.append(idx_tgts)
            
        traj_op_tgts = torch.tensor(traj_op_tgts)
        traj_tok_tgts = torch.tensor(traj_tok_tgts)
        traj_idx_tgts = torch.tensor(traj_idx_tgts)
        
        with open(f'../data/imdb/ezpz/sup-imdb-1_{i}.zst', 'wb') as f:
            pickle.dump((traj_op_tgts, traj_tok_tgts, traj_idx_tgts), f)

In [ ]:
import torch.nn.functional as F

traj_op_tgts = F.one_hot(traj_op_tgts, 5)
traj_tok_tgts = F.one_hot(traj_tok_tgts, VOCAB_SIZE)
traj_idx_tgts = F.one_hot(traj_idx_tgts, 512)

In [ ]:
from data import elaborate

elaborate((traj_op_tgts, traj_tok_tgts, traj_idx_tgts))